In [2]:
import trimesh
import numpy as np
import os

def load_3d_model(file_path):
    """Load a 3D model file and return a trimesh mesh."""
    try:
        mesh = trimesh.load(file_path)
        return mesh
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

def voxelize_mesh(mesh, resolution):
    """Convert the mesh into voxels at the specified resolution."""
    voxels = mesh.voxelized(pitch=resolution)
    return voxels

def generate_build_commands(voxels, scale=1):
    """Generate building commands from voxel data."""
    points = voxels.points
    commands = []
    current_pos = [0, 0, 0]
    
    # Sort points by z, then y, then x for optimal building
    sorted_points = sorted(points, key=lambda p: (p[1], p[2], p[0]))
    
    for point in sorted_points:
        # Calculate shifts needed with scaling
        shifts = []
        for i, (curr, target) in enumerate(zip(current_pos, point)):
            # Apply scaling to the movement
            scaled_target = target * scale
            scaled_curr = curr * scale
            diff = int(scaled_target - scaled_curr)
            if diff != 0:
                axis = ['x', 'y', 'z'][i]
                shifts.append(f"/shift {axis} {diff}")
                current_pos[i] = target
        
        # Add shifts and place block
        commands.extend(shifts)
        commands.append("/s default:stone")
    
    return commands

def save_commands(commands, output_file):
    """Save the generated commands to one or multiple files if exceeding 30,000 lines."""
    MAX_LINES = 30000
    
    # Split into multiple files if needed
    if len(commands) > MAX_LINES:
        base_name, ext = os.path.splitext(output_file)
        num_files = (len(commands) + MAX_LINES - 1) // MAX_LINES
        
        for i in range(num_files):
            start_idx = i * MAX_LINES
            end_idx = min((i + 1) * MAX_LINES, len(commands))
            current_file = f"{base_name}_part{i+1}{ext}"
         
            with open(current_file, 'w') as f:
                for cmd in commands[start_idx:end_idx]:
                    f.write(cmd + '\n')
            print(f"Part {i+1} saved to: {current_file}")
    else:
        # Single file if under limit
        with open(output_file, 'w') as f:
            for cmd in commands:
                f.write(cmd + '\n')
        print(f"Commands saved to: {output_file}")

def main():
    # Get input from user
    model_path = input("Enter the path to your 3D model file: ")
    resolution = float(input("Enter the desired block size (resolution, smaller = more detail): "))
    scale = int(input("Enter the scale factor (1 = normal size, 2 = double size, etc.): "))
    filename = input("Enter the output file name (e.g., build_commands.txt): ")
    
    # Create output path in user's home directory
    home_dir = os.path.expanduser("~")
    output_file = os.path.join(home_dir, filename)
    print(f"File will be saved to: {output_file}")
    
    # Load and process the model
    mesh = load_3d_model(model_path)
    if mesh is None:
        return
    
    # Convert to voxels
    print("Converting model to blocks...")
    voxels = voxelize_mesh(mesh, resolution)
    
    # Generate commands
    print("Generating building commands...")
    commands = generate_build_commands(voxels, scale=scale)
    
    # Save commands
    save_commands(commands, output_file)
    print(f"Total number of blocks: {len(commands)}")

if __name__ == "__main__":
    main()

File will be saved to: /Users/pougetfernet/heh.txt
Converting model to blocks...
Generating building commands...
Part 1 saved to: /Users/pougetfernet/heh_part1.txt
Part 2 saved to: /Users/pougetfernet/heh_part2.txt
Part 3 saved to: /Users/pougetfernet/heh_part3.txt
Part 4 saved to: /Users/pougetfernet/heh_part4.txt
Part 5 saved to: /Users/pougetfernet/heh_part5.txt
Total number of blocks: 125188
